<a href="https://colab.research.google.com/github/ss010101/IITRPRAI_24082382/blob/main/Abhman1minigpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi
!pip -q install "transformers>=4.41" peft accelerate bitsandbytes \
                 sentence-transformers faiss-cpu pypdf duckduckgo-search

/bin/bash: line 1: nvidia-smi: command not found
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')
WORKDIR = "/content/drive/MyDrive/colab_rag_smallllm"
import os; os.makedirs(WORKDIR, exist_ok=True)

Mounted at /content/drive


In [3]:
BASE_LLM = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"   # or "HuggingFaceTB/SmolLM-360M-Instruct"
EMB_MODEL = "intfloat/multilingual-e5-small"
CONTEXT_LEN = 1024
TOP_K = 6

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

tok = AutoTokenizer.from_pretrained(BASE_LLM, use_fast=True)
if tok.pad_token is None: tok.pad_token = tok.eos_token

llm = AutoModelForCausalLM.from_pretrained(
    BASE_LLM, device_map="auto", load_in_4bit=True
)

emb = SentenceTransformer(EMB_MODEL, device="cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [5]:
import os, glob, re
from pypdf import PdfReader

DOC_DIR = f"{WORKDIR}/docs"
os.makedirs(DOC_DIR, exist_ok=True)

def read_file(path):
    if path.lower().endswith(".pdf"):
        text = []
        reader = PdfReader(path)
        for p in reader.pages: text.append(p.extract_text() or "")
        return "\n".join(text)
    else:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()

def chunk(text, size=700, overlap=120):
    # simple sentence-ish splitter + sliding window
    text = re.sub(r"\s+", " ", text).strip()
    chunks = []
    start = 0
    while start < len(text):
        end = min(start+size, len(text))
        # try to end on punctuation
        cut = text.rfind(".", start, end)
        if cut == -1: cut = end
        chunks.append(text[start:cut].strip())
        start = max(cut - overlap, start + size)
    return [c for c in chunks if len(c) > 40]

docs = []
for p in glob.glob(f"{DOC_DIR}/**/*", recursive=True):
    if os.path.isfile(p) and any(p.lower().endswith(ext) for ext in [".pdf",".txt",".md",".html",".csv",".json",".log",".py",".ipynb"]):
        t = read_file(p);
        for c in chunk(t):
            docs.append({"text": c, "source": os.path.basename(p)})
len(docs)

0

In [6]:
import faiss, numpy as np, pickle, os, math

texts = [d["text"] for d in docs]
if not texts:
    texts = ["(placeholder) add files into /docs first"]
    docs = [{"text":texts[0], "source":"placeholder"}]

def embed_batch(strs, bs=64):
    vecs = []
    for i in range(0, len(strs), bs):
        v = emb.encode(strs[i:i+bs], normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=False)
        vecs.append(v)
    return np.vstack(vecs)

X = embed_batch(texts)
index = faiss.IndexFlatIP(X.shape[1])     # cosine sim since normalized
index.add(X.astype(np.float32))

with open(f"{WORKDIR}/kb_meta.pkl","wb") as f:
    pickle.dump({"docs": docs}, f)
faiss.write_index(index, f"{WORKDIR}/kb.index")
print("KB saved:", len(docs), "chunks")

KB saved: 1 chunks


In [7]:
import pickle, faiss, numpy as np
from duckduckgo_search import DDGS

# load KB
kb = pickle.load(open(f"{WORKDIR}/kb_meta.pkl","rb"))
index = faiss.read_index(f"{WORKDIR}/kb.index")
docs = kb["docs"]

def retrieve(query, k=TOP_K):
    qv = emb.encode([query], normalize_embeddings=True, convert_to_numpy=True)
    D, I = index.search(qv.astype(np.float32), k)
    hits = []
    for score, idx in zip(D[0], I[0]):
        if idx == -1: continue
        hits.append({"score": float(score), **docs[idx]})
    return hits

def web_search(query, max_results=5):
    out = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=max_results, safesearch="moderate"):
            out.append({"title": r.get("title",""), "href": r.get("href",""), "body": r.get("body","")})
    return out

In [8]:
import textwrap, torch
from transformers import TextStreamer

SYSTEM = "You are a concise, bilingual (EN/JP) assistant. Cite sources inline like [S1], [W2]. If unsure, say so."

def make_context(query, local_hits, web_hits):
    ctx_lines, citations = [], []
    # local first
    for i,h in enumerate(local_hits,1):
        tag = f"S{i}"
        ctx_lines.append(f"[{tag}] Source: {h['source']}\n{h['text']}\n")
        citations.append(tag)
    # web snippets
    for j,w in enumerate(web_hits,1):
        tag = f"W{j}"
        snippet = (w["title"]+": " if w["title"] else "") + (w["body"] or "")
        ctx_lines.append(f"[{tag}] Web: {w.get('href','')}\n{snippet}\n")
        citations.append(tag)
    ctx = "\n".join(ctx_lines[:12])
    return ctx

def generate_rag(query, k_local=TOP_K, use_web=True, web_k=5, thresh=0.2, max_new=300):
    local_hits = retrieve(query, k_local)
    # simple confidence check: mean similarity
    conf = np.mean([h["score"] for h in local_hits]) if local_hits else 0.0
    web_hits = web_search(query, web_k) if (use_web and conf < thresh) else []

    context = make_context(query, local_hits, web_hits)
    prompt = (
        f"<s>[SYSTEM]\n{SYSTEM}\n[/SYSTEM]\n"
        f"[USER]\nQuestion: {query}\n\nContext:\n{context}\n"
        f"Instructions:\n- Answer using the context when relevant.\n"
        f"- Include inline citations [S#]/[W#].\n- If context is insufficient, say what’s missing.\n[/USER]\n[ASSISTANT]\n"
    )

    streamer = TextStreamer(tok, skip_prompt=True, skip_special_tokens=True)
    ids = tok(prompt, return_tensors="pt", truncation=True, max_length=CONTEXT_LEN).to(llm.device)
    out = llm.generate(**ids, max_new_tokens=max_new, temperature=0.3, top_p=0.9, do_sample=True, streamer=streamer)
    return ""

In [9]:
_ = generate_rag("京都で1日観光するならおすすめルートは？")
# Try a fresh/current event to see web fallback:
_ = generate_rag("Tokyo weather today")

Sure, here's an answer using the context:

1. Day tour in Kyoto:
- If you're looking for a day tour that covers the most popular sights in Kyoto, the 

KeyboardInterrupt: 